In [ ]:
print('Tales')
print('Lero')

<h1> UNILAB 2018 - Discentes, docentes, disciplinas e cursos 


Membros:
*   Lucas Brant - 
*   Luiz Felipe Nery - 2018054770 
*   Pedro Wildemberg - 
*   Tales Panoutsos - 

<br>

---



<h1>  Descrição dos dados</h1>
<p>Bla bla</p>
<br>



<h1>Diagrama ER</h1>

<h1>Diagrama Relacional</h1>
<h2> Forma Normalizada

* **cursos**(id_curso, nome_curso, nivel, modalidade, area_conhecimento)

* **docentes**(id_docente, nome_docente, sigla_unidade, situacao)

  docente.sigla_unidade -> unidade.sigla

* **unidades**(sigla, nome_unidade)

* **discentes**(id_discente, num_matricula, ano_ingresso, periodo_atual, sexo, idade, raca, pais, forma_ingresso, discente_situacao, beneficiado_auxilio, id_curso)

* **matriculas**(id_matricula_disciplina, resultado_disciplina, ano, periodo, id_disciplina, id_discente, media_final, numero_faltas)

  matricula.id_discente -> discente.id_discente

  matricula.id_docente -> docente.id_docente

  matricula.id_disciplina -> disciplina.id_disciplina

* **disciplinas**(id_disciplina, nome_disciplina, carga_horaria, id_docente)

  disciplinas.id_docente -> docentes.id_docentes


<br>

---


<h1>Baixando os dados

<br>
<p>Todos os dados são referentes ao ano de 2018

In [ ]:
#matriculas
#Modelo:     !wget  -O .csv
!wget http://dadosabertos.unilab.edu.br:8001/csv/matriculas_2018.csv -O matriculas_2018.csv
!wget http://dadosabertos.unilab.edu.br:8001/csv/lista_discentes.csv -O alunos.csv
!wget http://dadosabertos.unilab.edu.br:8001/csv/cursos_graduacao.csv -O cursos.csv
!wget http://dadosabertos.unilab.edu.br:8001/csv/docentes.csv -O docentes.csv


In [ ]:
!wget -c "http://dadosabertos.unilab.edu.br/api/action/datastore_search?resource_id=89132ea3-318d-44e1-a269-2795ed168ae8&limit=1600" -O  disciplinas.json

In [ ]:
!ls

--- 

<br>
<h1>Adicionando bibliotecas


In [ ]:
import io
import sqlite3
import pandas as pd
import numpy as np

<br>

---

<br>

<h1>Editando os dados
<br>

<br>

<h2>Disciplinas

In [ ]:
#json cru, obtido da requisicao
disciplinas = pd.read_json("disciplinas.json")

#Selecionamos somente os resultados na coluna "result" linha "records"
disciplinas = pd.DataFrame.from_dict(disciplinas["result"]["records"], orient='columns')

disciplinas = disciplinas.drop("unidade",1)
disciplinas = disciplinas.drop("curso",1)
disciplinas = disciplinas.drop("horario",1)
disciplinas = disciplinas.drop("vagas",1)
disciplinas = disciplinas.drop("_id",1)
disciplinas = disciplinas.drop("local",1)
disciplinas = disciplinas.drop("id_unidade",1)

#cada disciplina so pode ser dada por um professor
disciplinas.drop_duplicates("codigo_disciplina", keep = 'first', inplace = True)

print(len(disciplinas))
disciplinas.head()

In [ ]:
disciplinas[disciplinas["codigo_disciplina"] == "CCAP0103"]

<br>

---

<br>

<h2>Matriculas</h2>





* **matriculas**(id_matricula_disciplina, resultado_disciplina, ano, periodo, id_disciplina, id_discente, media_final, numero_faltas, docente_nome)

  matricula.id_discente -> discente.id_discente

  matricula.id_docente -> docente.id_docente

  matricula.id_disciplina -> disciplina.id_disciplina

In [ ]:
matriculas_denorm = pd.read_csv("matriculas_2018.csv")

matriculas = matriculas_denorm[['id_matricula_disciplina', 'resultado_disciplina', 'ano', 'periodo', 'codigo_disciplina', 'id_discente', 'media_final', 'numero_faltas']].copy()
print('Número de linhas: ',len(matriculas))
matriculas.drop_duplicates(inplace = True)
matriculas.head(6)

<br><h3>Consistente?</h3>

<p>Possui dulicatas?<p>

In [ ]:
matriculas[['codigo_disciplina','id_discente','periodo', 'ano','resultado_disciplina','media_final','numero_faltas', 'id_matricula_disciplina']].duplicated().any()

<br><p>Possui chave?</p>
<p>Sim, a coluna id_matricula_disciplina</p><br>


<br>

---

<br>

<h2> Cursos

In [ ]:
cursos = pd.read_csv("cursos.csv")
cursos = cursos.drop("Tipo de Oferta da Disciplina",1)
cursos = cursos.drop("Portaria Reconhecimento",1)
cursos = cursos.drop("DOU",1)
cursos = cursos.drop("Data Cadastro",1)
cursos = cursos.drop("Grau Acadêmico",1)
cursos = cursos.drop("Código INEP",1)
cursos = cursos.drop(20) #Retirando coluna que repetida
cursos = cursos.drop("Município",1)
print("Numero de de cursos: " + str(len(cursos)))
print("Tem duplicata usando Cursos e Modalidade Educação como chave? " + str((cursos[["Curso","Modalidade Educação"]]).duplicated().any()))
cursos.head()

In [ ]:
#Removendo as duplicatas
cursos.drop_duplicates(subset =["Curso","Modalidade Educação"],keep = 'first', inplace = True)
print("Tem duplicata usando Cursos e Modalidade Educação como chave? " + str((cursos[["Curso","Modalidade Educação"]]).duplicated().any()))
print("Numerode de cursos: " + str(len(cursos)))

O objetivo de nao haver duplicatas em cursos em relacao ao nome do curso e a modalidade de educação é porque dessa forma podemos fazer um mapeamento na tabela alunos para o ID de cada curso. Dessa forma, fazermos isso para deixar as tabelas na forma normal e evitar redundancias. 

<br>

---

<br>

<h2> Docentes

In [ ]:
docentes = pd.read_csv("docentes.csv")
docentes = docentes.drop('categoria',1)
docentes = docentes.drop('denominacao',1)
docentes = docentes.drop('unidade',1)
print(len(docentes))
docentes.head() 

In [ ]:
print("Tamanho de docentes: " + str(len(docentes)))
print("Tamanho de docentes sem repetir nomes: " + str(len(docentes.groupby(["nome"]).count())))

# Sabemos que nome não é uma chave primaria muito boa, contudo, como nao temos as informacoes necessarias nas outras tabelas em que um docente aparece, nao conseguimos trocar a chave primaria para uma chave unica, por exemplo, uma numerica.

<br>

---

<br>

<h2> Alunos

In [ ]:
alunos = pd.read_csv("alunos.csv")
alunos.groupby('estado').count()

In [ ]:
#tem q tirar os alunos do para
#todos eles tem o curso bugado: SAÚDE DA FAMÍLIA - 2017.2
((alunos[alunos['estado'] == 'Pará']).groupby("curso").count()).head()

#Observe que todos os alunos do Pará fazem o emso curso. Este curso não esta na tabela de cursos.

In [ ]:
#Todos os cursos que sao feitos por esses alunos -> Muitos deles são cursos antigos (de antes de 2018) que não estao 
#em nossa tabela de cursos, uma vez que temos somente informações sobre os cursos de 2018
print("numero de cursos: " + str(len(alunos.groupby('curso').count().index)))
alunos.groupby('curso').count().index

In [ ]:
alunos = pd.read_csv("alunos.csv")
discentes = pd.read_csv("alunos.csv")

discentes = discentes.drop('polo',1)
discentes = discentes.drop('prazo_conclusao',1)
discentes = discentes.drop('periodo_ingresso',1)
discentes = discentes.drop('data_colacao_grau',1)
discentes = discentes.drop('nivel',1)
discentes = discentes.drop('estado',1)

In [ ]:
curso_id = []
for i in range (len(alunos)):
  match = False
  for j in range (len(cursos)):
    if alunos["curso"].iloc[i] == cursos["Curso"].iloc[j] and alunos["modalidade"].iloc[i] == cursos["Modalidade Educação"].iloc[j]:
      id = cursos["ID"].iloc[j]
      curso_id.append(id)
      match = True
  if not(match):
    discentes = discentes.drop(alunos.index[i])

In [ ]:
discentes["id_curso"] = curso_id

In [ ]:
discentes = discentes.drop('curso',1)
discentes = discentes.drop('modalidade',1)

In [ ]:
discentes.head()

<br>
<br>

---

<br>

<h1> Criando o banco de dados</h1>

**'df' deve ser usada como variavel temporaria**

In [ ]:
!rm unilab.sqlite
!ls

In [ ]:
#criando e conectando banco
conn = sqlite3.connect("unilab.sqlite") #se o arquivo nao existir, um novo e gerado
cursor = conn.cursor()
conn.execute("PRAGMA foreign_keys = 1") #garante que cada foreign key existe na tabela referenciada, se nao existir a insercao da erro
!ls

---

<br>

<h2>Criando tabela Unidades no bando de dados 

In [ ]:
#criando tabela UNIDADE
cursor.execute('''CREATE TABLE UNIDADE
         (sigla TEXT PRIMARY KEY NOT NULL,
         unidade TEXT NOT NULL);''')

In [ ]:
#inserindo dados na tabela unidade
df = pd.read_csv('docentes.csv')
df = df[['sigla', 'unidade']].drop_duplicates() #tuplas tem de ser unicas
df.to_sql('UNIDADE', conn, if_exists='append', index=False)
unidades_sql = df.copy()

In [ ]:
unidades_sql

---

<br>

<h2> Criando tabela Docente no banco de dados

In [ ]:
#criando tabela DOCENTE
cursor.execute('''CREATE TABLE DOCENTE
         (nome TEXT NOT NULL,
         situacao TEXT NOT NULL,
         sigla TEXT NOT NULL,
         PRIMARY KEY (nome, sigla),
         FOREIGN KEY (sigla)
          REFERENCES UNIDADE(sigla));''')

#coloquei sigla e nome como chave primaria, aparentemente tem professores com nomes iguais em institutos diferentes

In [ ]:
#Inserindo dados na tabela DOCENTE
df = pd.read_csv('docentes.csv')
df = df[['nome', 'sigla', 'situação']].drop_duplicates()
df.columns = ['nome', 'sigla', 'situacao'] #troquei os nomes do dataframe pra ficar igual da tabela
df.to_sql('DOCENTE', conn, if_exists='append', index=False) #if_exists=append garante que vai inserir na tabela existente
                                                             #index=False evita que uma coluna adicional seja criada 

In [ ]:
docentes_sql = df.copy()
docentes_sql

<h3>Consulta de teste

In [ ]:
#query de exemplo com os dados acima
df = pd.read_sql("SELECT * FROM DOCENTE WHERE sigla = 'ICS'", conn)
df

---

<br>

<h2> Criando tabela Cursos no banco de dados

In [ ]:
#criando tabela CURSOS

cursor.execute('''CREATE TABLE CURSOS 
         (id_curso INTEGER PRIMARY KEY NOT NULL, 
         nome_curso TEXT NOT NULL, 
         modalidade TEXT NOT NULL, 
         area_conhecimento TEXT NOT NULL); ''')

#coloquei sigla e nome como chave primaria, aparentemente tem professores com nomes iguais em institutos diferentes

In [ ]:
df = cursos.copy()
if('Município' in df.columns):
  df = df.drop('Município',1)
print(df.columns)
df.columns = ['id_curso', 'nome_curso', 'modalidade', 'area_conhecimento'] #troquei os nomes do dataframe pra ficar igual da tabela
print(df.columns)

In [ ]:
#Inserindo dados na tabela CURSOS
df.to_sql('CURSOS', conn, if_exists='append', index=False) #if_exists=append garante que vai inserir na tabela existente
                                                             #index=False evita que uma coluna adicional seja criada 

<h3> Consulta Teste

In [ ]:
df = pd.read_sql("SELECT * FROM CURSO WHERE area_conhecimento = 'Ciências Sociais Aplicadas'", conn)
df

---

<br>

<h1>Consultas</h1>
<h2>Consultas usando Projeção e seleção </h2>
<br>
<h3>Consulta 1

<h3>Consulta 2

---

<br>

<h2>Consultas envolvendo junção de duas relações 
<br>
<h3>Consulta 1 

<h3>Consulta 2

<h3>Consulta 3

---

<br>

<h2>Consultas envolvendo junção de três ou mais relações

<br>

<h3>Consulta 1

<h3>Consulta 2

<h3>Consulta 3

---

<br>

<h2>Consultas envolvendo agregação sobre junção de duas ou mais relações

<br>

<h3>Consulta 1


<h3>Consulta 2

---

<br>

<h1>Autoavaliação dos membros</h1>








# Divisão quanto ao tratamento de dados:
Luiz: Docentes e disciplinas

Lucas: alunos

Tales: matriculas

Pedro: Cursos

<br>
<br>

Obs1.: Os nomes acima seguem o padrao do dataframe que pode ser vista abaixo

Obs2.: Mantenham o padrão dos nomes

Obs3.: Esse notebook será o entregavel final, então a medida que foram fazendo as coisas vao documentando para ficar mais facil

Obs4.: Se nao for possivel alterar esse notebook no original, salvem ele e trabalhem separadamente e depois a gente junta tudo